In [13]:
import sys
sys.path.append('../scripts')
from iat_predictor_train_and_eval import get_embeddings_and_labels, add_vis_embs

import json
import pandas as pd

In [22]:
import numpy as np
import pandas as pd

def get_embeddings_and_labels_multiple(annotations_df, procedures_df, tasks_df, col, model_name, current_fps, target_fps):
    """
    Generates a DataFrame with embeddings and labels, expanding rows for each individual vision embedding.

    This function takes annotations and optional procedure/task dataframes. For each row 
    in the annotations_df, it duplicates the row for every frame-wise vision embedding 
    found in 'vis_embs_multiple'. The final DataFrame contains a row for each individual 
    frame, with a new 'vis_embs_individual' column and a combined 'embedding' column.
    
    Vision embeddings can be downsampled from a `current_fps` to a `target_fps`.

    Args:
        annotations_df (pd.DataFrame): DataFrame containing annotations and a column with lists of vision embeddings.
        procedures_df (pd.DataFrame): Optional DataFrame with procedure text embeddings.
        tasks_df (pd.DataFrame): Optional DataFrame with task text embeddings.
        col (str): The name of the column in annotations_df to use as the label.
        model_name (str): The name of the model used for embeddings, to determine which columns to use.
        current_fps (int): The original frames per second of the vision embeddings.
        target_fps (int): The target frames per second to sample the embeddings to.

    Returns:
        pd.DataFrame: A new DataFrame where each row corresponds to a single vision embedding,
                      containing the individual vision embedding, its corresponding label, any text
                      embeddings, and a final concatenated embedding vector.
    """
    df = annotations_df.copy()
    
    # Vision embeddings are expected to be a list of embedding arrays for each row.
    vis_embs_lists = df[f'{model_name}_vis_embs'].values.tolist()
    
    # Sample the vision embeddings based on target FPS.
    if current_fps > target_fps:
        stride = int(round(current_fps / target_fps))
        if stride > 1:
            print(f"Sampling embeddings from {current_fps}fps to {target_fps}fps with a stride of {stride}.")
            vis_embs_lists = [embs[::stride] for embs in vis_embs_lists]
    
    # Determine the corresponding text embedding column based on the model name.
    emb_col = ''
    if model_name == 'surgvlp': emb_col = 'SurgVLP'
    elif model_name == 'hecvl': emb_col = 'HecVL'
    elif model_name == 'peskavlp': emb_col = 'PeskaVLP'
    elif 'pe' in model_name or 'videomae' in model_name: emb_col = 'MedEmbed_small'
    else:
        raise ValueError(f"Model name {model_name} is not supported.")
    print(f"Embedding column: {emb_col}")
    
    # Extract procedure embeddings if the dataframe is provided.
    procedure_embs = []
    if procedures_df is not None:
        for i in range(len(df)):
            case = df.iloc[i]['case']
            tmp_df = procedures_df[procedures_df['case_id'] == case]
            procedure_embs.append(tmp_df[f"procedure_defn_emb-{emb_col}"].values[0])
    
    # Extract task embeddings if the dataframe is provided.
    task_embs = []
    if tasks_df is not None:
        if 'secs' not in df.columns:
             df['secs'] = df['cvid'].apply(lambda x: sum([a*b for a,b in zip(map(int, x.split('_')[-1][:-4].split('-')), [3600, 60, 1])]))
        
        for i in range(len(df)):
            secs = df.iloc[i]['secs']
            case = df.iloc[i]['case']
            tmp_df = tasks_df[tasks_df['case_id'] == case]
            tmp_df = tmp_df[(tmp_df['start_secs'] <= secs) & (tmp_df['end_secs'] > secs)]
            emb_size = len(tasks_df[f"task_defn_emb-{emb_col}"].iloc[0])
            if len(tmp_df) == 0:
                task_embs.append(np.zeros(emb_size))
            elif len(tmp_df) > 1:
                print(f"Multiple teaching steps found for case {case} and timestamp {secs}")
                task_embs.append(np.zeros(emb_size))
            else:
                task_embs.append(tmp_df[f"task_defn_emb-{emb_col}"].values[0])
    
    # Extract labels.
    labels = df[col].values.tolist()
    
    # --- Main Logic: Expand the DataFrame ---
    # Create a new list of rows, where each original row is duplicated for each of its vision embeddings.
    expanded_rows = []
    for i in range(len(df)):
        original_row_data = df.iloc[i].to_dict()
        current_label = labels[i]
        current_proc_emb = procedure_embs[i] if procedures_df is not None else None
        current_task_emb = task_embs[i] if tasks_df is not None else None
        
        # Iterate over each individual vision embedding for the current row.
        for vis_emb in vis_embs_lists[i]:
            new_row = original_row_data.copy()
            
            # Add the specific vision embedding for this new row.
            new_row['vis_embs_individual'] = vis_emb
            new_row['label'] = current_label

            # Add procedure and task embeddings.
            if procedures_df is not None:
                new_row['procedure_embs'] = current_proc_emb
            if tasks_df is not None:
                new_row['task_embs'] = current_task_emb

            # Create the final combined embedding.
            embs_to_concat = [vis_emb]
            if procedures_df is not None:
                embs_to_concat.append(current_proc_emb)
            if tasks_df is not None:
                embs_to_concat.append(current_task_emb)
            
            new_row['embedding'] = np.concatenate(embs_to_concat)
            expanded_rows.append(new_row)
            
    final_df = pd.DataFrame(expanded_rows).reset_index(drop=True)

    # Print embedding dimensions for verification.
    print(f"Vision Embedding Dimension: {len(final_df['vis_embs_individual'].iloc[0])}")
    if procedures_df is not None:
        print(f"Procedure Embedding Dimension: {len(final_df['procedure_embs'].iloc[0])}")
    if tasks_df is not None:
        print(f"Task Embedding Dimension: {len(final_df['task_embs'].iloc[0])}")
    print(f"Combined Embedding Dimension: {len(final_df['embedding'].iloc[0])}")
    
    return final_df



In [10]:
task_normalization_json = json.load(open('/home/firdavs/surgery/surgical_fb_generation/SurgFBGen/data/iat/task_normalization.json'))
annotations_df = pd.read_csv('/home/firdavs/surgery/surgical_fb_generation/SurgFBGen/data/iat_predictor_splits/full.csv').assign(task=lambda x: x['task'].map(task_normalization_json))

procedures_df = pd.read_parquet('/home/firdavs/surgery/surgical_fb_generation/SurgFBGen/data/iat/procedures_embs_df.parquet')
tasks_df = pd.read_parquet('/home/firdavs/surgery/surgical_fb_generation/SurgFBGen/data/iat/tasks_embs_df.parquet')

In [14]:
annotations_df = add_vis_embs(
    name='peskavlp', 
    annotations_df=annotations_df, 
    embedding_dir='/home/firdavs/surgery/surgical_fb_generation/SurgFBGen/outputs/embeddings/vision',
)

In [23]:
df = get_embeddings_and_labels_multiple(
    annotations_df=annotations_df,
    procedures_df=procedures_df,
    tasks_df=tasks_df,
    col='instrument',
    model_name='peskavlp',
    current_fps=5,
    target_fps=1
)

Sampling embeddings from 5fps to 1fps with a stride of 5.
Embedding column: PeskaVLP
Multiple teaching steps found for case 1 and timestamp 4384
Multiple teaching steps found for case 14 and timestamp 2871
Multiple teaching steps found for case 28 and timestamp 7038
Multiple teaching steps found for case 28 and timestamp 7119
Multiple teaching steps found for case 28 and timestamp 7152
Multiple teaching steps found for case 28 and timestamp 7210
Multiple teaching steps found for case 28 and timestamp 7010
Multiple teaching steps found for case 28 and timestamp 7031
Multiple teaching steps found for case 28 and timestamp 7102
Multiple teaching steps found for case 1 and timestamp 4374
Vision Embedding Dimension: 768
Procedure Embedding Dimension: 768
Task Embedding Dimension: 768
Combined Embedding Dimension: 2304


In [24]:
df

,dialogue,timestamp,case,cvid,procedure,procedure_defn,task,task_defn,instrument,action,...,instrument-extraction,action-extraction,tissue-extraction,peskavlp_vis_embs,secs,vis_embs_individual,label,procedure_embs,task_embs,embedding
0,you can even come 1 mm closer to the prostate,09:14:19,1,c1_s0_0-36-54.avi,radical prostatectomy,Radical prostatectomy is a surgical procedure ...,Bladder Neck,The bladder neck is the region where the bladd...,NONE,NONE,...,[NONE],[come closer],[prostate],"[[0.50458145, -1.1343791, -0.3422805, 0.167872...",2214,"[0.50458145, -1.1343791, -0.3422805, 0.1678728...",NONE,"[-0.5317898, -0.120111674, -0.18510374, -0.517...","[-0.54922867, -0.19645786, 0.037684545, -0.571...","[0.50458145, -1.1343791, -0.3422805, 0.1678728..."
1,you can even come 1 mm closer to the prostate,09:14:19,1,c1_s0_0-36-54.avi,radical prostatectomy,Radical prostatectomy is a surgical procedure ...,Bladder Neck,The bladder neck is the region where the bladd...,NONE,NONE,...,[NONE],[come closer],[prostate],"[[0.50458145, -1.1343791, -0.3422805, 0.167872...",2214,"[0.55762535, -0.68891823, -0.45461792, 0.24765...",NONE,"[-0.5317898, -0.120111674, -0.18510374, -0.517...","[-0.54922867, -0.19645786, 0.037684545, -0.571...","[0.55762535, -0.68891823, -0.45461792, 0.24765..."
2,you can even come 1 mm closer to the prostate,09:14:19,1,c1_s0_0-36-54.avi,radical prostatectomy,Radical prostatectomy is a surgical procedure ...,Bladder Neck,The bladder neck is the region where the bladd...,NONE,NONE,...,[NONE],[come closer],[prostate],"[[0.50458145, -1.1343791, -0.3422805, 0.167872...",2214,"[0.4300177, -0.38983494, -0.29604316, -0.14735...",NONE,"[-0.5317898, -0.120111674, -0.18510374, -0.517...","[-0.54922867, -0.19645786, 0.037684545, -0.571...","[0.4300177, -0.38983494, -0.29604316, -0.14735..."
3,you can even come 1 mm closer to the prostate,09:14:19,1,c1_s0_0-36-54.avi,radical prostatectomy,Radical prostatectomy is a surgical procedure ...,Bladder Neck,The bladder neck is the region where the bladd...,NONE,NONE,...,[NONE],[come closer],[prostate],"[[0.50458145, -1.1343791, -0.3422805, 0.167872...",2214,"[0.56297106, -1.1774668, -0.28983954, 0.361305...",NONE,"[-0.5317898, -0.120111674, -0.18510374, -0.517...","[-0.54922867, -0.19645786, 0.037684545, -0.571...","[0.56297106, -1.1774668, -0.28983954, 0.361305..."
4,you can even come 1 mm closer to the prostate,09:14:19,1,c1_s0_0-36-54.avi,radical prostatectomy,Radical prostatectomy is a surgical procedure ...,Bladder Neck,The bladder neck is the region where the bladd...,NONE,NONE,...,[NONE],[come closer],[prostate],"[[0.50458145, -1.1343791, -0.3422805, 0.167872...",2214,"[0.5119209, -0.56926906, -0.53749424, 0.240415...",NONE,"[-0.5317898, -0.120111674, -0.18510374, -0.517...","[-0.54922867, -0.19645786, 0.037684545, -0.571...","[0.5119209, -0.56926906, -0.53749424, 0.240415..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14495,come back look to the right,15:34:28,8,c8_s0_0-2-9.avi,nephrectomy,Nephrectomy is a surgical procedure that invol...,Dissection,Dissecting involves carefully separating tissu...,NONE,navigate_space,...,[NONE],"[come back, look to the right]",[NONE],"[[0.17421353, -0.15229097, -0.04908724, 0.0480...",129,"[-0.00830745, -0.17275849, 0.22704677, -0.3761...",NONE,"[-0.37363967, -0.41328382, -0.18113622, 0.1482...","[-0.074749865, 0.10011853, 0.28964067, -0.3364...","[-0.00830745, -0.17275849, 0.22704677, -0.3761..."
14496,come back look to the right,15:34:28,8,c8_s0_0-2-9.avi,nephrectomy,Nephrectomy is a surgical procedure that invol...,Dissection,Dissecting involves carefully separating tissu...,NONE,navigate_space,...,[NONE],"[come back, look to the right]",[NONE],"[[0.17421353, -0.15229097, -0.04908724, 0.0480...",129,"[-0.16521557, -0.13561733, 0.33576015, -0.0841...",NONE,"[-0.37363967, -0.41328382, -0.18113622, 0.1482...","[-0.074749865, 0.10011853, 0.28964067, -0.3364...","[-0.16521557, -0.13561733, 0.33576015, -0